Sistema de recomendación

Importar librerias

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, date
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

Concatenar nuevamente los df de ratings

In [148]:
df_rating1 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/1.csv', sep = ',')
df_rating2 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/2.csv', sep = ',')
df_rating3 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/3.csv', sep = ',')
df_rating4 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/4.csv', sep = ',')
df_rating5 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/5.csv', sep = ',')
df_rating6 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/6.csv', sep = ',')
df_rating7 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/7.csv', sep = ',')
df_rating8 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/8.csv', sep = ',')

In [149]:
for ind, time in enumerate(df_rating1.timestamp):
    df_rating1['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating2.timestamp):
    df_rating2['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating3.timestamp):
    df_rating3['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating4.timestamp):
    df_rating4['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating5.timestamp):
    df_rating5['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating6.timestamp):
    df_rating6['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating7.timestamp):
    df_rating7['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating8.timestamp):
    df_rating8['timestamp'][ind] = date.fromtimestamp(time)

C:\Users\Carlo\AppData\Local\Temp\ipykernel_2656\4204255159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating1['timestamp'][ind] = date.fromtimestamp(time)
C:\Users\Carlo\AppData\Local\Temp\ipykernel_2656\4204255159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating2['timestamp'][ind] = date.fromtimestamp(time)
C:\Users\Carlo\AppData\Local\Temp\ipykernel_2656\4204255159.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [150]:
df_ratings = pd.concat([df_rating1, df_rating2, df_rating3, df_rating4, df_rating5, df_rating6, df_rating7, df_rating8], ignore_index=True).drop_duplicates()

In [151]:
df_movies = pd.read_csv('Ratings.csv', sep = ';')

In [152]:
df_ratings.head()

,userId,rating,timestamp,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500


In [153]:
df_movies.head()

,id,title,listed_in,rating
0,as1,the grand seduction,"comedy, drama",3.467131
1,as2,take care good night,"drama, international",3.548682
2,as3,secrets of deception,"action, drama, suspense",3.498812
3,as4,pink: staying true,documentary,3.537076
4,as5,monster maker,"drama, fantasy",3.478992


Del df_movie, usar el id y title.

In [154]:
df_movies = df_movies[['id', 'title']] #Es para configurar los datos de entrenamiento.

In [155]:
df_movies = df_movies.reset_index(drop=True)

Antes de entrenar el modelo, elegir la cantidad de datos a usar en el modelo. Es decir, nuestro modelo será un modelo supervisado.

In [156]:
num_rows = 120000 #No usar todos los datos. Usar cantidad pertienente para predecir con el modelo.
reader = Reader(rating_scale = (1, 5))
dates = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']][:num_rows], reader)
trainset, testset = train_test_split(dates, test_size = .20) #Usar el 80% de los datos para el entrenamiento del modelo.

Entrenar el modelo.

In [157]:
model = SVD()
model.fit(trainset)

In [158]:
predicts = model.test(testset)

In [159]:
model.predict(154,'as680')

Prediction(uid=154, iid='as680', r_ui=None, est=3.88329300853749, details={'was_impossible': False})

In [168]:
#Evaluo el modelo 
from surprise import accuracy

accuracy.rmse(predicts)

RMSE: 0.9597


0.959740508784853

In [169]:
#Optimizo hiperparametros
from surprise.model_selection import cross_validate

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, dates, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9461  0.9491  0.9546  0.9499  0.0035  
Fit time          0.64    0.58    0.55    0.59    0.04    
Test time         0.21    0.60    0.19    0.33    0.19    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9516  0.9472  0.9503  0.9497  0.0018  
Fit time          0.56    0.55    0.56    0.56    0.00    
Test time         0.18    0.42    0.19    0.26    0.11    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9548  0.9463  0.9504  0.9505  0.0035  
Fit time          0.57    0.58    0.60    0.58    0.01    
Test time         0.18    0.18    0.19    0.18    0.00    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9551  0.9

In [190]:
def score_recommendation(userId: int, movieId: str):
    prediction = model.predict(userId, movieId) # Obtenemos la predicción de la calificación que el usuario le daría a la película
    if prediction.est >= 3.5: #Si la predicción es mayor o igual a 3.5, se recomienda la película
        return "Recomendada", prediction.est
    else:
        return "No recomendada", prediction.est

In [196]:
def titulo(movieId: str):
    return df_movies[df_movies.id == movieId].title.iloc[0].title()

In [197]:
def recommendation(userId: int, movieId: str):
    recomendado, puntaje = score_recommendation(userId, movieId)
    titulo_1 = titulo(movieId)
    return (recomendado, puntaje, titulo_1)

In [198]:
recommendation(548,"ds568")

('No recomendada', 3.0596348515292897, 'A Tale Of Two Critters')

In [204]:
import gradio as gra

def score_recommendation(userId: int, movieId: str):
    prediction = model.predict(userId, movieId) # Obtenemos la predicción de la calificación que el usuario le daría a la película
    if prediction.est >= 3.5: #Si la predicción es mayor o igual a 3.5, se recomienda la película
        return "Recomendada", prediction.est
    else:
        return "No recomendada", prediction.est
    
def titulo(movieId: str):
    return df_movies[df_movies.id == movieId].title.iloc[0].title()

def recommendation(userId: int, movieId: str):
    recomendado, puntaje = score_recommendation(userId, movieId)
    titulo_1 = titulo(movieId)
    return (recomendado, puntaje, titulo_1)

app = gra.Interface(fn = recommendation, inputs= [int, str], outputs=[str, float, str])
app.launch()

ImportError: cannot import name errors